# Нейронка класс своё MNIST

## Загрузка библиотек

In [ ]:
import torch
import torchvision

from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader, random_split

import os
import json
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

import struct
import sys
import random

from os import path
from array import array

from torchvision.transforms import v2

from torchvision import transforms 
from tqdm import tqdm

from matplotlib.ticker import AutoMinorLocator, MultipleLocator

from torch import nn
import time

## Архитектура нейронной сети

In [ ]:
class MyNN(nn.Module):
    def __init__(self, input_, output):
        super().__init__()
        self.layer1 = nn.Linear(input_, 128)
        self.layer2 = nn.Linear(128, output)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.act(x)
        out = self.layer2(x)
        return out

# Обязательная ячейка

In [ ]:
path_to_all_data = r'C:\Users\user\Desktop\learn models'

## Определение устройства

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Обработка изображений

In [ ]:
transform = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=(0.5, ), std=(0.5, ))    
])

## Класс датасета

In [ ]:
class MNISTDataset(Dataset): # собственный класс для датасета
    def __init__(self, path, transform=None):
        self.path = path
        self.transform = transform
        self.len_dataset = 0
        self.data_list = [] # (путь до файла, позиция класса в ван хот векторе)
        
        for path_dir, dir_list, file_list in os.walk(path):
            if path_dir == path:
                self.classes = sorted(dir_list)
                self.class_to_idx = {cls_name: i for i, cls_name in enumerate(self.classes)}
                continue
            cls = path_dir.split('\\')[-1]
            for name_file in file_list:
                file_path = os.path.join(path_dir, name_file)
                self.data_list.append((file_path, self.class_to_idx[cls]))
            self.len_dataset += len(file_list)
    
    def __len__(self):
        return self.len_dataset
    
    def __getitem__(self, index):
        file_path, target = self.data_list[index]
        sample = Image.open(file_path)
        if self.transform is not None:
            sample = self.transform(sample)
        return sample, target

## Подгрузка датасета

In [ ]:
train_data = MNISTDataset(os.path.join(path_to_all_data, r'MNIST\data\training'), transform=transform)
test_data = MNISTDataset(os.path.join(path_to_all_data, r'MNIST\data\testing'), transform=transform)
for_one_hot_vector = MNISTDataset(os.path.join(path_to_all_data, r'MNIST\data\training'), transform=transform)

train_data, val_data = random_split(train_data, [0.7, 0.3])

# батчи
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

## Модель, гипер, шедуллер

In [ ]:
model = MyNN(784, 10).to(device)

loss_model = nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.001) # lr - скорость обучения

lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, # оптимизатор
                                                          mode='min', # max/min- следит чтобы отслеживаемый параметр увеличивался уменьшался
                                                          factor=0.1, # коэф на который будет умножен lr
                                                          patience=4, # кол-во эпох без улучшения отслеживаемого параметра
                                                          threshold=0.0001, # порог на которой должен имениться отслеживаемый параметр
                                                          threshold_mode='rel', # rel / abs. rel -отслеживаемый параметр должен измениться на threshhold% иначе просто
                                                          cooldown=0, # кол-во перодов ожидания после уменьшения lr
                                                          min_lr=0, # минимальное значение скорости обучения
                                                          eps=1e-8 # минимальное изменение между новым и старым lr
                                                         )

## Предтрен списки

In [ ]:
EPOCHS = 50
best_loss, train_loss, train_acc, val_loss, val_acc, lr_list, treshold = None, [], [], [], [], [], 0.00001
count = 0 # число итераций без улучшения модели

std_info = '''
class MyNN(nn.Module):
    def __init__(self, input_, output):
        super().__init__()
        self.layer1 = nn.Linear(input_, 128)
        self.layer2 = nn.Linear(128, output)
        self.act = nn.ReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.act(x)
        out = self.layer2(x)
        return out
        '''

## Обучение

In [ ]:
%%time

# Цикл обучения
#     Тренировка модели
#         Данные     
#         Прямой проход и расчёт ошибки модели    
#         Обратный проход
#         Шаг оптимизации
#     Расчёт метрики
#     Сохранение лоса и метрики

#     Валидация
#         Данные     
#         Прямой проход и расчёт ошибки модели            
#     Расчёт метрики
#     Сохранение лоса и метрики

# цикл обучения
for epoch in range(EPOCHS):

    # тренировка модели
    model.train()
    running_train_loss = []
    true_answer = 0 # для подсчёта правильных ответов
    train_loop = tqdm(train_loader, leave=False) # для когздания прогресс бара
    for x, targets in train_loop:
        # данные (batch_size, 1, 28, 28) -> (batch_size, 784)
        x = x.reshape(-1, 28 * 28).to(device)
        # (batch_size, int) -> (batch_size, 10), dtype=float32
        targets = targets.reshape(-1).to(torch.int32) # делаем одномерный массив из нужных значений
        targets = torch.eye(10)[targets].to(device) # из единичной матрицы 10*10 выдергиваем необходимые строки в соответствии с таргет

        # прямой проход и расчет ошибки модели
        pred = model(x)
        loss = loss_model(pred, targets)

        # обратный проход
        opt.zero_grad() # обнуляем раннее вычесленный градиент
        loss.backward() # производится обратный проход в результате которого получаются новые градиенты 
        # шаг оптимизации
        opt.step() # корректировка весов

        running_train_loss.append(loss.item()) # item - возвращает только значения функции потерь
        mean_train_loss = sum(running_train_loss) / len(running_train_loss)

        true_answer += (pred.argmax(dim=1) == targets.argmax(dim=1)).sum().item()
        
        train_loop.set_description(f'Epoch [{epoch+1}/{EPOCHS}], train_loss={mean_train_loss:.4f}')

    
    # расчёт значений метрики
    running_train_acc = true_answer / len(train_data)
    #сохранение значения функции потерь и метрики
    train_loss.append(mean_train_loss)
    train_acc.append(running_train_acc)

    # проверка модели - валидация
    model.eval()
    with torch.no_grad(): # запрещаем вычисление градиента
        running_val_loss = []
        true_answer = 0
        for x, targets in val_loader:
            # данные (batch_size, 1, 28, 28) -> (batch_size, 784)
            x = x.reshape(-1, 28 * 28).to(device)
            # (batch_size, int) -> (batch_size, 10), dtype=float32
            targets = targets.reshape(-1).to(torch.int32)
            targets = torch.eye(10)[targets].to(device)        


            # прямой проход и расчет ошибки модели
            pred = model(x)
            loss = loss_model(pred, targets)

            running_val_loss.append(loss.item())
            mean_val_loss = sum(running_val_loss) / len(running_val_loss)
            # val_loop.set_description(f'Epoch [{epoch+1}/{EPOCHS}], val_loss={mean_val_loss:.4f}')

            true_answer += (pred.argmax(dim=1) == targets.argmax(dim=1)).sum().item()
        
        # расчёт значений метрики
        running_val_acc = true_answer / len(val_data)
        #сохранение значения функции потерь и метрики
        val_loss.append(mean_val_loss)
        val_acc.append(running_val_acc)

        # шаг шедуллера
        lr_scheduler.step(mean_val_loss)
        lr_list.append(lr_scheduler._last_lr[0])

        print(f'Epoch [{epoch+1}/{EPOCHS}], train_loss={mean_train_loss:.4f}, train_acc={running_train_acc:.4f}, val_loss={mean_val_loss:.4f}, val_acc={running_val_acc:.4f}')

        if best_loss is None:
            best_loss = mean_val_loss

        if mean_val_loss < best_loss - best_loss * treshold:
            count = 0
            best_loss = mean_val_loss
            # torch.save(model.state_dict(), f'model_mnist/model_state_dict_epoch_{epoch}_mnist.pt')
            checkpoint = {
                        'class_to_idx': for_one_hot_vector.class_to_idx,
                        'info': std_info,
                        'state_model': model.state_dict(),
                        'state_opt': opt.state_dict(),
                        'state_lr_scheduler': lr_scheduler.state_dict(),
                        'loss': {
                            'train_loss': train_loss,
                            'val_loss': val_loss,
                            'best_loss': best_loss
                        },
                        'metric': {
                            'train_acc': train_acc,
                            'val_acc': val_acc
                        },
                        'lr': lr_list,
                        'epoch': {
                            'EPOCHS': EPOCHS,
                            'save_epoch': epoch
                        }
                    }
            torch.save(checkpoint, f'model_mnist/model_state_mnist_{epoch}_checkpoint.pt')
            for i in os.listdir('model_mnist'):
                if i == f'model_state_dict_epoch_{epoch}_mnist.pt' or i == f'model_mnist/model_state_mnist_{epoch}__checkpoint.pt':
                    continue
                os.remove(os.path.join(r'model_mnist', i))
            print(f'На эпохе {epoch+1}, сохранена модель со значением функции потерь на валидации - {mean_val_loss:.4f}', end='\n\n')

        if count >= 10:
            print(f"\033[31mОбучение остановленно на {epoch + 1} эпохе.\033[0m")
            break
        count += 1

In [ ]:
len(train_data)